In [1]:
import sys 
import os 

In [2]:
from datetime import datetime, timedelta
from dateparser import parse

In [3]:
from pathlib import Path
from subprocess import call

In [4]:
import shutil

In [5]:
HOME = Path.home()
CWD = Path.cwd()

### first clean up the repository 

In [6]:
git = shutil.which("git")

In [7]:
git

'/usr/bin/git'

In [8]:
str(CWD.parent.joinpath("GPM_IMERG"))

'/home/nicolasf/operational/ICU_Web/GPM_IMERG'

In [9]:
cmd = f"{git} rm -r {str(CWD.parent.joinpath('GPM_IMERG'))}/*"

In [10]:
call(cmd, shell=True)

0

In [11]:
cmd = f"{git} rm -r {str(CWD.parent.joinpath('C3S'))}/*"

In [12]:
call(cmd, shell=True)

128

### small utility to clean up country names 

In [13]:
def sanitize_name(name): 
    """
    Sanitizes the name of a country 

    removes all the weird characters 

    Parameters
    ----------
    name : str
        country name 

    Returns
    -------
    str
        The Sanitized country name
    """
    name = name.replace('/','')
    name = name.replace('  ','_')
    name = name.replace('&','')
    name= name.replace(' ','_')
    name = name.replace(':','_')
    name = name.replace('__','_')
    name = name.replace("'","")
    return name

### some definitions 

In [14]:
days = [30, 60, 90, 180, 360]

### list of countries 

In [15]:
list_countries = \
['Kiribati_Line_Islands',
 'Fiji',
 'New_Caledonia',
 'Papua_New_Guinea',
 'Wallis_Futuna',
 'Cook_Islands',
 'Solomon_Islands',
 'French_Polynesia',
 'Palau',
 'Tuvalu',
 'Vanuatu',
 'Pitcairn_Islands',
 'American_Samoa',
 'Northern_Marianas',
 'Niue',
 'Guam',
 'Samoa',
 'Federated_States_of_Micronesia',
 'Nauru',
 'Kiribati_Phoenix_Islands',
 'Tokelau',
 'Kiribati_Gilbert_Islands',
 'Tonga',
 'Marshall_Islands']

### general path for the GPM-IMERG figures 

In [16]:
fig_path_GPM = HOME.joinpath('operational/ICU/development/hotspots/figures/GPM_IMERG')

### dictionnary mapping destination folder to search pattern for the Pacific domain maps 

In [17]:
dsearch = {}
dsearch['SPI'] = 'GPM_IMERG_SPI_Pacific'
dsearch['EAR_Watch'] = 'EAR_Watch_Pacific'
dsearch['USDM'] = 'USDM_Pacific'
dsearch['accumulations'] = 'precip_accumulation'
dsearch['anomalies'] = 'precip_anomalies'
dsearch['dry_days'] = 'nb_dry_days'
dsearch['last_rain'] = 'days_since_rain'

In [18]:
for k in dsearch.keys(): 
    
    opath = CWD.parent.joinpath(f"GPM_IMERG/{k}/Pacific/")
    
    if not(opath.exists()): 
    
        opath.mkdir(parents=True)
    
    print(f"finding files for {k}")
    
    list_files = list(fig_path_GPM.glob(f"{dsearch[k]}*"))
    
    list_files.sort()
    
    print(f"found {len(list_files)} for variable {k}")
    
    last_date = list_files[-1].name.split("_")[-1].split(".")[-2]
    
    last_date = datetime.strptime(last_date, "%Y-%m-%d")
    
    print(f"last date is {last_date:%Y-%m-%d}")
    
    list_files = list(fig_path_GPM.glob(f"{dsearch[k]}*{last_date:%Y-%m-%d}.png"))
    
    print(f"will now copy {len(list_files)} files")
    
    for fname in list_files: 
    
        filename = fname.name.replace(f"_to_{last_date:%Y-%m-%d}", "_latest")
    
        dest = opath.joinpath(f"{filename}")
        
        # dest.unlink(missing_ok=True)
    
        shutil.copy(fname, dest)
    
    print(f"copied \n{chr(10).join(map(str,list_files))}")

finding files for SPI
found 61 for variable SPI
last date is 2022-01-08
will now copy 5 files
copied 
/home/nicolasf/operational/ICU/development/hotspots/figures/GPM_IMERG/GPM_IMERG_SPI_Pacific_90days_to_2022-01-08.png
/home/nicolasf/operational/ICU/development/hotspots/figures/GPM_IMERG/GPM_IMERG_SPI_Pacific_30days_to_2022-01-08.png
/home/nicolasf/operational/ICU/development/hotspots/figures/GPM_IMERG/GPM_IMERG_SPI_Pacific_180days_to_2022-01-08.png
/home/nicolasf/operational/ICU/development/hotspots/figures/GPM_IMERG/GPM_IMERG_SPI_Pacific_60days_to_2022-01-08.png
/home/nicolasf/operational/ICU/development/hotspots/figures/GPM_IMERG/GPM_IMERG_SPI_Pacific_360days_to_2022-01-08.png
finding files for EAR_Watch
found 115 for variable EAR_Watch
last date is 2022-01-16
will now copy 5 files
copied 
/home/nicolasf/operational/ICU/development/hotspots/figures/GPM_IMERG/EAR_Watch_Pacific_60days_to_2022-01-16.png
/home/nicolasf/operational/ICU/development/hotspots/figures/GPM_IMERG/EAR_Watch_Pac

### Now process the figures for one country 

In [19]:
for country_name in list_countries: 
    
    for k in ['SPI', 'EAR_Watch', 'USDM']: 
        
        list_files = list(fig_path_GPM.joinpath(f"{k}").glob(f"{country_name}_mask_EEZ_{k}*_to_{last_date:%Y-%m-%d}.png"))
        
        for fname in list_files: 

            filename = fname.name.replace(f"_to_{last_date:%Y-%m-%d}", "_latest")
            
            dest_dir = CWD.parent.joinpath(f"GPM_IMERG/{k}/{country_name}")
            
            if not(dest_dir.exists()): 
                
                dest_dir.mkdir(parents=True)

            dest = dest_dir.joinpath(f"{filename}")
            
            dest.unlink(missing_ok=True)

            shutil.copy(fname, dest)

        print(f"copied \n{chr(10).join(map(str,list_files))}")

copied 
/home/nicolasf/operational/ICU/development/hotspots/figures/GPM_IMERG/SPI/Kiribati_Line_Islands_mask_EEZ_SPI_90nbdays_to_2022-01-08.png
/home/nicolasf/operational/ICU/development/hotspots/figures/GPM_IMERG/SPI/Kiribati_Line_Islands_mask_EEZ_SPI_360nbdays_to_2022-01-08.png
/home/nicolasf/operational/ICU/development/hotspots/figures/GPM_IMERG/SPI/Kiribati_Line_Islands_mask_EEZ_SPI_30nbdays_to_2022-01-08.png
/home/nicolasf/operational/ICU/development/hotspots/figures/GPM_IMERG/SPI/Kiribati_Line_Islands_mask_EEZ_SPI_60nbdays_to_2022-01-08.png
/home/nicolasf/operational/ICU/development/hotspots/figures/GPM_IMERG/SPI/Kiribati_Line_Islands_mask_EEZ_SPI_180nbdays_to_2022-01-08.png
copied 
/home/nicolasf/operational/ICU/development/hotspots/figures/GPM_IMERG/EAR_Watch/Kiribati_Line_Islands_mask_EEZ_EAR_Watch_360nbdays_to_2022-01-08.png
/home/nicolasf/operational/ICU/development/hotspots/figures/GPM_IMERG/EAR_Watch/Kiribati_Line_Islands_mask_EEZ_EAR_Watch_60nbdays_to_2022-01-08.png
/home

## updating the github repository 

In [20]:
cmds = [f"{git} add {str(CWD.parent.joinpath('GPM_IMERG'))}/*",\
       f"{git} commit -m 'updated to {last_date:%Y-%m-%d}'",\
       f"{git} push"]

In [21]:
for cmd in cmds: 
    r = call(cmd, shell=True)
    print(f"return code for {cmd}: {r}")

return code for /usr/bin/git add /home/nicolasf/operational/ICU_Web/GPM_IMERG/*: 0
return code for /usr/bin/git commit -m 'updated to 2022-01-08': 0
return code for /usr/bin/git push: 0


In [22]:
cmds = [f"{git} add {str(CWD.parent.joinpath('C3S'))}/*",\
       f"{git} commit -m 'updated to {last_date:%Y-%m-%d}'",\
       f"{git} push"]

In [23]:
for cmd in cmds: 
    call(cmd, shell=True)
    print(f"return code for {cmd}: {r}")

return code for /usr/bin/git add /home/nicolasf/operational/ICU_Web/C3S/*: 0
return code for /usr/bin/git commit -m 'updated to 2022-01-08': 0
return code for /usr/bin/git push: 0
